# Packages

In [1]:
from quocslib.optimalcontrolproblems.OneQubitProblem_2fields import OneQubit2Fields
import time, datetime

import scipy
import numpy as np
from scipy import linalg
import os
import sys
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.optimize import minimize
import time
import copy
from scipy import signal
# import necessary stuff from quidi
from logic.pulsed.pulse_objects import PulseBlock, PulseBlockEnsemble, PulseSequence

import logging
logging.basicConfig(filename='logfile.log', filemode='w', level=logging.DEBUG)
logger = logging.getLogger(__name__)

0

0

# Library functions

In [2]:

def dict_2_header_str(in_dict):
    out_str = ""
    for key, val in in_dict.items():
        out_str += f"{key}: {val}\n"
        
    return out_str

def get_pulse_filename(path, name="", name_ampl="amplitude.txt", name_phase="phase.txt"):
    return os.path.abspath(path + "/" + name + name_ampl), os.path.abspath(path + "/" + name + name_phase)

def save_pulse(path, data_ampl, data_phase, name="", name_ampl="amplitude.txt", name_phase="phase.txt"):
    header_dict = {}
    try:
        now = datetime.datetime.now() # current date and time
        header_dict['timestamp'] = now.strftime("%Y/%m/%d-%H:%M:%S")
        header_dict['script_parameters'] = script_params
        header_dict.update(opti_comm_dict['optimization_dictionary'])
        
        fname_ampl, fname_phase = get_pulse_filename(path, name=name, name_ampl=name_ampl, name_phase=name_phase)
        
        if data_ampl is not None:
            np.savetxt(fname_ampl,  data_ampl,  header=dict_2_header_str(header_dict))
        if data_phase is not None:
            np.savetxt(fname_phase, data_phase, header=dict_2_header_str(header_dict))
    except:
        logger.exception("")

        
def save_optimization_result(path, name="opt_res_", name_ampl="amplitude.txt", name_phase="phase.txt"):
    try:
        optimizer_obj = optimizationlogic.optimization_obj
        best_dict = optimizer_obj.opt_alg_obj.get_best_controls()
        
        pulses_list = best_dict['pulses']
        time_grids_list = best_dict['timegrids']
        parameters_list = best_dict['parameters']
        
        t_amplitude = time_grids_list[0]
        amplitude = pulses_list[0]
        t_phase = time_grids_list[1]
        phase = pulses_list[1]

        data_ampl = np.column_stack((t_amplitude, amplitude))
        data_phase = np.column_stack((t_phase, phase))
        fom = np.column_stack((range(len(fom_all)), fom_all))

        save_pulse(path, data_ampl, data_phase, name=name, name_ampl=name_ampl, name_phase=name_phase)
        # save fom by same function as pulse
        save_pulse(path, fom, None, name=name, name_ampl='fom.txt', name_phase='')
    except:
        logger.exception("")
#save_pulse(folder_path, np.asarray([1,2]), np.asarray(1))
#get_pulse_filename(folder_path, name=filename)

# QuOCS and Qudi with Noise

## Creation of the optimization dictionary

The optimization dictionary contains all the settings compulsory for the optimization algorithm in order to run a proper optimization.

In [3]:
optimization_client_name = "test_dCRAB_Noisy_2_control_fields"

optimization_dictionary = {"optimization_client_name": optimization_client_name,
                           'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABNoisyAlgorithm', 
                           'opti_algorithm_class': 'DCrabNoisyAlgorithm', 
                          }

Number of iteration and super-iterations

In [4]:
# Total number of dCRAB superiteration
super_iteration_number = 10000000
# Maximum number of iteration per super-iteration (in max_eval_total: maximale anzahl von evaluation steps gesamt)
maximum_function_evaluations_number = 500

To activate the drift compensation and the re-evaluation steps add

In [5]:
optimization_dictionary['algorithm_settings'] = {"algorithm_name": "dCRAB",
                                                 "super_iteration_number": super_iteration_number,
                                                 #"max_eval_total": super_iteration_number * maximum_function_evaluations_number,
                                                 "FoM_goal": 0.00001,
                                                 "total_time_lim": 48*60,  # minutes
                                                 "compensate_drift": {
                                                     "compensate_after_SI": True,
                                                     "compensate_after_minutes": 15
                                                 },
                                                 "random_number_generator":{
                                                     "seed_number":420
                                                 },
                                                 #"re_evaluation": {
                                                     #"re_evaluation_steps": [0.3, 0.5, 0.51]
                                                 #}
}

Settings for the inner algorithm used by dCRAB

In [6]:
optimization_dictionary['algorithm_settings']["dsm_settings"] = {'general_settings': {
                                                "dsm_algorithm_name": "NelderMead", 
                                                'is_adaptive': False
                                            }, 
                                            'stopping_criteria': {
                                                #"max_eval": 100,
                                                "time_lim": 90,  #min
                                                #"xatol": 1e-10, 
                                                #"frtol": 1e-10,
                                                "change_based_stop": {
                                                    "cbs_funct_evals": 500, 
                                                    "cbs_change": 0.01    # stop if slope below
                                                }
                                            }
}
optimization_dictionary['algorithm_settings']

{'algorithm_name': 'dCRAB', 'super_iteration_number': 10000000, 'FoM_goal': 1e-05, 'total_time_lim': 2880, 'compensate_drift': {'compensate_after_SI': True, 'compensate_after_minutes': 15}, 'random_number_generator': {'seed_number': 420}, 'dsm_settings': {'general_settings': {'dsm_algorithm_name': 'NelderMead', 'is_adaptive': False}, 'stopping_criteria': {'time_lim': 90, 'change_based_stop': {'cbs_funct_evals': 500, 'cbs_change': 0.01}}}}

### Times

In [10]:
t_rabi_rect = 50e-9  
pix_pulse = 1
t_oc_pulse = 100e-9 
fac_oc_guess = 1.0 # make initial guess not too good, otherwise hard for optimizer

ampl_max = 0.25  # V
ampl_rect = 0.24  # V # todo: double check that (V) and not normed from 0 to 1


fac_oc_rect = t_oc_pulse/(0.5*t_rabi_rect/pix_pulse)
if fac_oc_rect  < 3:
    logger.warning(f"OC pulse should be >3x as long as rect, now: {t_oc_pulse/(0.5*t_rabi_rect/pix_pulse)}")

time_p = {'time_name': 'time_p', 'initial_value': t_oc_pulse} # pulse length
optimization_dictionary["times"] = [time_p]
ampl_oc_guess = fac_oc_guess*1/fac_oc_rect*ampl_rect

ampl_oc_guess, fac_oc_rect

(0.06, 4.0)

### Parameters

In [11]:
optimization_dictionary['parameters'] = []

### Pulses

In [12]:
vec_number = 2   #2-3
upper_freq_lim = 10  #8


    
pulse_amplitude = {'pulse_name': 'Amplitude', 
                           'upper_limit':ampl_max, 
                           'lower_limit': -ampl_max, 
                           'bins_number': 1001, 
                           'time_name': 'time_p', 
                           'amplitude_variation': 3*ampl_oc_guess, # added to guess amp, to create 3 points for start simplex
                           'basis': {'basis_name': 'Fourier', 
                                     'basis_class': 'Fourier', 
                                     'basis_module': 'quocslib.pulses.basis.Fourier', 
                                     'basis_vector_number': vec_number, # number of frequencies within the below defined range
                                     'random_super_parameter_distribution': 
                                     {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                                           'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                                           'lower_limit': 0.0, 'upper_limit': upper_freq_lim} # number of oscillations within pulse
                                    }, 
                           'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
                           'initial_guess': {'function_type': 'lambda_function', 'lambda_function': f'lambda t: {ampl_oc_guess} + 0.0*t'}
                          }

pulse_phase = {'pulse_name': 'Phase', 
               'upper_limit': ampl_max, 
               'lower_limit': -ampl_max, 
               'bins_number': 1001, 
               'time_name': 'time_p', 
               'amplitude_variation': 3*ampl_oc_guess, # added to guess amp, to create 3 points for start simplex
               'basis': {'basis_name': 'Fourier', 
                         'basis_class': 'Fourier', 
                         'basis_module': 'quocslib.pulses.basis.Fourier', 
                         'basis_vector_number': vec_number, # number of frequencies within the below defined range
                         'random_super_parameter_distribution': 
                         {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                               'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                               'lower_limit': 0.0, 'upper_limit': upper_freq_lim} # number of oscillations within pulse
                        }, 
               'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
               'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 0.0*t'}
              }

optimization_dictionary['pulses'] = [pulse_amplitude, pulse_phase]

ampl_oc_guess

0.06

### Put all together and get ready to start the optimization with Qudi-QuOCS

In [14]:
opti_comm_dict = {"optimization_dictionary": optimization_dictionary}
optimizationlogic.load_opti_comm_dict(opti_comm_dict)

### Test before run


Press "Start" in the gui and see whether errors are logged.

Print the optimization dictionary also here. If the GUI is not showing the optimization dictionary, restart the Kernel.

In [15]:
optimalcontrol.opti_comm_dict

{'optimization_dictionary': {'optimization_client_name': 'test_dCRAB_Noisy_2_control_fields', 'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABNoisyAlgorithm', 'opti_algorithm_class': 'DCrabNoisyAlgorithm', 'algorithm_settings': {'algorithm_name': 'dCRAB', 'super_iteration_number': 10000000, 'FoM_goal': 1e-05, 'total_time_lim': 2880, 'compensate_drift': {'compensate_after_SI': True, 'compensate_after_minutes': 15}, 'random_number_generator': {'seed_number': 420}, 'dsm_settings': {'general_settings': {'dsm_algorithm_name': 'NelderMead', 'is_adaptive': False}, 'stopping_criteria': {'time_lim': 90, 'change_based_stop': {'cbs_funct_evals': 500, 'cbs_change': 0.01}}}}, 'times': [{'time_name': 'time_p', 'initial_value': 1e-07}], 'parameters': [], 'pulses': [{'pulse_name': 'Amplitude', 'upper_limit': 0.25, 'lower_limit': -0.25, 'bins_number': 1001, 'time_name': 'time_p', 'amplitude_variation': 0.18, 'basis': {'basis_name': 'Fourier', 'basis_class': 'Fourier', 'basis_module': 'quocsli

# Measurement

Run a normal rect Rabi with all analysis and MW generation parameters before. Then run with commented out the dicts below in order to keep all the respective pulsed settings.

In [16]:
script_params = {}
script_params['runtime_per_epoch'] = 40
script_params['readout_noise'] = 0.01  # std_dev of fom, controls convergence in iteration
# needs to be smaller than the actual pulse length time_p due to the interpolation
script_params['length_oc'] = t_oc_pulse - 1e-9
script_params['is_noisy'] = True

# file paths 
folder_path = r'C:\Software\qudi_data\optimal_control_assets'
filename= "on_nv=2_"

## Set measurement sequence and fom

In [17]:
def setup_sts_oc_pi(folder_path, filename, length_oc):
    dictparamsall = pulsedmasterlogic.generate_method_params
    
    dict_qst = dictparamsall["sts_oc"]
    dict_qst["experiment"] = 'sts_oc'
    dict_qst["name"] = 'stsoc'
    dict_qst["length"] = length_oc
    dict_qst["filename_amplitude"] = os.path.basename(get_pulse_filename(folder_path, name=filename)[0])
    dict_qst["filename_phase"] = os.path.basename(get_pulse_filename(folder_path, name=filename)[1])
    dict_qst["folder_path"] = folder_path
    

    
    return dict_qst

def setup_oc_npi(folder_path, filename, length_oc, n_start=1, n_step=1, n_n=4, t_gap=0e-9):
    dictparamsall = pulsedmasterlogic.generate_method_params
    
    dict_qst = dictparamsall["oc_nrep"]
    dict_qst["experiment"] = 'oc_nrep'
    dict_qst["name"] = 'oc_nrep'
    dict_qst["filename_amplitude"] = os.path.basename(get_pulse_filename(folder_path, name=filename)[0])
    dict_qst["filename_phase"] = os.path.basename(get_pulse_filename(folder_path, name=filename)[1])
    dict_qst["folder_path"] = folder_path
    
    dict_qst['n_start'] = n_start
    dict_qst['n_step'] = n_step
    dict_qst['t_gap'] = t_gap
    dict_qst['num_of_points'] = n_n
    
    dict_qst["alternating"] = True
    dict_qst["vs_rect_pulse"] = False
    
    
    return dict_qst

def setup_fom_func_sts_oc_pi():
    def calc_fom():
        signal_data = pulsedmeasurementlogic.signal_data
        up_norm = signal_data[1,0] / signal_data[1,0]
        down_norm = signal_data[1,1] / signal_data[1,0]

        return np.real(1 - (up_norm - down_norm))
    
    return calc_fom

def setup_fom_func_oc_npi():
    def calc_fom():
        # signal_data[0,:] = xaxis
        # signal_data[1,:] = yaxis (alternating flattened)
        try:
            signal_data = pulsedmeasurementlogic.signal_data
            n_pi = signal_data[0,:]
            n_pi_even = np.where(n_pi%2==0)

            data_mw = signal_data[1,:]
            data_alt = signal_data[2,:]
            data_norm = data_mw/data_alt
            data_mw_0 = (data_norm)[n_pi%2==0]  # normed in state |0>
            data_mw_1 = (data_norm)[n_pi%2==1]  # normed in state |1>

            return np.mean(1 - (data_mw_0 - data_mw_1))
        except:
            return np.nan
    
    return calc_fom

qmeas = setup_sts_oc_pi(folder_path, filename, script_params['length_oc'])
qmeas = setup_oc_npi(folder_path, filename, script_params['length_oc'])
qmeas = setup_oc_npi(folder_path, filename, script_params['length_oc'], t_gap=0.25/2.1e6)
fom_func = setup_fom_func_oc_npi()
fom_func = setup_fom_func_oc_npi()

script_params['generate_parameters'] = qmeas
script_params['fom_function'] = fom_func

fom_func(), qmeas

Traceback (most recent call last):
  File "C:\Software\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-74c66498f215>", line 69, in <module>
    qmeas = setup_oc_npi(folder_path, filename, script_params['length_oc'])
  File "<ipython-input-17-74c66498f215>", line 19, in setup_oc_npi
    dict_qst = dictparamsall["oc_nrep"]
KeyError: 'oc_nrep'


In [783]:
signal_data = pulsedmeasurementlogic.signal_data
n_pi = signal_data[0,:]
data_mw = signal_data[1,:]
data_alt = signal_data[2,:]
data_norm = data_mw/data_alt
data_mw_0 = (data_norm)[n_pi%2==0]
data_mw_1 = (data_norm)[n_pi%2==1]

data_mw_1

array([0.90143771, 0.9131982 ])

## Run

In [789]:
script_params['generate_parameters']['experiment']

'oc_nrep'

In [ ]:
######################################################################################################
# Parameters and Settings
######################################################################################################

# runtime of each experiment
runtime = script_params['runtime_per_epoch']
length_oc = script_params['length_oc'] 

# parameter to stop the experiment if its set to False in the console
pulsedmasterlogic.globalrun = True
uglobals.abort.clear()
logger.info("Starting new closed-loop quocs optimization")

# Dictonary containing the general measurement parameters for the predefined measurements
p_generation_dict = pulsedmasterlogic.generation_parameters
#p_generation_dict["laser_channel"] = "d_ch2" 
#p_generation_dict["sync_channel"] = "d_ch1" 
#p_generation_dict["gate_channel"] = None#"d_ch1" 
#p_generation_dict["microwave_channel"] = "a_ch1"
#p_generation_dict["microwave_amplitude"] = 0.25
#p_generation_dict["microwave_frequency"] = 2822.3e6
#p_generation_dict["rabi_period"] = 82.7e-9
#p_generation_dict["laser_length"] = 3e-06
#p_generation_dict["laser_delay"] = 0.2e-6
#p_generation_dict["wait_time"] = 1e-06
#p_generation_dict["analog_trigger_voltage"] = 0.0
# load the general measurement parameters for the predefined measurements
pulsedmasterlogic.set_generation_parameters(p_generation_dict)
script_params['pulsed_generation_params'] = pulsedmasterlogic.generation_parameters


# tell the measurement gui how the sequence looks
pulsed_settings = dict()
pulsed_settings['invoke_settings'] = True
pulsedmasterlogic.set_measurement_settings(pulsed_settings)
script_params['pulsed_settings'] = pulsed_settings


# make sure everything is finished
time.sleep(5)

# how the laser response is analyzed    
#pulsedmasterlogic.set_analysis_settings(method = 'mean')
#pulsedmasterlogic.set_analysis_settings(signal_start = 150e-9)
#pulsedmasterlogic.set_analysis_settings(signal_end = 1e-6)
#pulsedmasterlogic.set_extraction_settings(method = 'conv_deriv')

p_analysis_settings = pulsedmasterlogic.analysis_settings
p_extraction_settings = pulsedmasterlogic.extraction_settings
script_params['pulsed_analysis_settings'] = p_analysis_settings
script_params['pulsed_extraction_settings_settings'] = p_extraction_settings

# array to save the fom evolution
fom_all = []

# This section is devoted to the initialization in the pulsed logic and optimization logic of the main
# settings and parameters to be usde in the creation ofthe pulse sequence and the optimization
# Iteration, controls and figure of merit to compare with QuOCS
# Just an example for debug
args_dict = {"is_noisy": script_params['is_noisy']}
qubit = OneQubit2Fields(args_dict)

######################################################################################################
# Measurement
######################################################################################################
optimalcontrol.start_optimization()

# crucial, maybe it can be much shorter
time.sleep(1)

# Just a time to check for latent time
last_time_fom = time.time()
# repeat the whole process until its manually stopped or QuOCS finsihed the optimization
# Wait few seconds before starting to get and return data
while not optimizationlogic.handle_exit_obj.is_user_running:
    time.sleep(0.1)
    if (time.time() - last_time_fom) > 30 or uglobals.abort.is_set():
        logger.warning("Timeout or abort while initializing quocs.")
        uglobals.abort.set()
        break

# iteration number
it_val = 0

# when did the optimization start?
opt_start_time = time.time()
try:
    # print("Check before the loop starts: {0}".format(optimizationlogic.handle_exit_obj.is_user_running))
    while  optimizationlogic.handle_exit_obj.is_user_running == True and not uglobals.abort.is_set():
        time_stamp=time.time()
        # wait until QuOCS optimizes the controls
        # print("Wait until the controls logic gives the controls")
        while not controlslogic.are_pulses_calculated:
            time.sleep(0.1)
            # If the waiting time exceed 10 seconds left stop the optimization
            if time.time() - last_time_fom > 20:
                logger.error("Too much time... Exit!")
                optimizationlogic.handle_exit_obj.is_user_running = False
                break

        # If time exceeded, exit from the while loop and give a very high fom to the optimization algorithm
        # and status code -1, to interrupt the optimization smoothly
        if optimizationlogic.handle_exit_obj.is_user_running == False:
            logger.info('Stopping the measurement!')
            # Update the pulse with the maximum number
            fomlogic.update_fom(10**10, status_code=-1)
            break

        #######################################################################################################
        # Get the Controls
        #######################################################################################################
        # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
        controlslogic.are_pulses_calculated = False
        # Get the controls from the controls logic
        pulses, parameters, timegrids = controlslogic.pulses, controlslogic.parameters, controlslogic.timegrids
        #######################################################################################################
        # Perform the measurement
        #######################################################################################################

        # save the pulses as .txt files (predefined methods doesn't allow us to upload a numpy array as 
        # parameter)
        data_ampl = np.column_stack((timegrids[0], pulses[0]))
        data_phase = np.column_stack((timegrids[0], pulses[1]))
        save_pulse(folder_path, data_ampl, data_phase, name=filename)

        time.sleep(0.2)
        
        qmeas = cp.deepcopy(script_params['generate_parameters'])
        experiment = qmeas['experiment']
        exp_name = qmeas['name']
        qmeas.pop('experiment', None)
        # generate the sequence
        pulsedmasterlogic.generate_predefined_sequence(experiment,kwarg_dict=qmeas)  

        time.sleep(1)

        # upload the sequence
        pulsedmasterlogic.sample_ensemble(exp_name,True)
        while pulsedmasterlogic.status_dict['sampload_busy'] or pulsedmasterlogic.status_dict['sampling_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:     
            time.sleep(1)

        #print('Finished uploading', dictparams_oc["name"], '!')

        # make sure everything is finished (crucial)
        #time.sleep(1)#time.sleep(5)    

         # start the measurement
        #print('Starting the measurement!')
        pulsedmasterlogic.toggle_pulsed_measurement(True)
        time.sleep(1)

        # make sure the measurement started
        while not pulsedmasterlogic.status_dict['measurement_running']:
            time.sleep(0.1)

        measurement_start_time = time.time()

        while time.time() <= measurement_start_time + runtime:
            time.sleep(0.2)

            # option to stop the measurement
            if pulsedmasterlogic.globalrun == False or uglobals.abort.is_set():
                print('Stopping the measurement!')
                break

        # option to stop the measurement
        if pulsedmasterlogic.globalrun == False or uglobals.abort.is_set():
            print('Stopping the measurement!')
            pulsedmasterlogic.toggle_pulsed_measurement(False)
            break

        #time.sleep(1)

        # Stop the measurement
        pulsedmasterlogic.toggle_pulsed_measurement(False)

        # Make sure it stopped (Wait until the Picoscope card sends the data)
        while pulsedmasterlogic.status_dict['measurement_running']:
            time.sleep(0.1)

        #######################################################################################################
        # Analysis
        ######################################################################################################

        fom_func = script_params['fom_function']
        fom = fom_func()
        # save the fom to plot its evolution later
        fom_all.append(fom)

        # Update the figure of merit and the standard deviation to the fom logic
        fomlogic.update_fom(fom, script_params['readout_noise'], status_code=0)   # todo: define above
        #fomlogic.update_fom(fom, std, status_code=0)

        # update the last time the fom is calculated
        last_time_fom = time.time()

    # when did the optimization stop?
    opt_end_time = time.time()

    print('It took QuOCS ' + str(opt_end_time-opt_start_time) + ' s to optimize the pulse!')
except:
    logger.exception("")
    uglobals.abort.set()
print("Optimization finished")

In [861]:
data_ampl

array([[0.00000000e+00, 1.84041210e-01],
       [1.60000000e-10, 2.10481705e-01],
       [3.20000000e-10, 2.35989645e-01],
       ...,
       [1.59680000e-07, 3.32141134e-02],
       [1.59840000e-07, 3.56110368e-02],
       [1.60000000e-07, 3.77439311e-02]])

## Test FOM noise

In [873]:
uglobals.abort.clear()

pulsed_settings = dict()
pulsed_settings['invoke_settings'] = True
pulsedmasterlogic.set_measurement_settings(pulsed_settings)
script_params['pulsed_settings'] = pulsed_settings


foms = []
n_mc = 20
idx= 0
try:
    while idx<n_mc and not uglobals.abort.is_set():
        time_stamp=time.time()

        qmeas = cp.deepcopy(script_params['generate_parameters'])
        experiment = qmeas['experiment']
        exp_name = qmeas['name']
        qmeas.pop('experiment', None)
        # generate the sequence
        pulsedmasterlogic.generate_predefined_sequence(experiment,kwarg_dict=qmeas)  

        time.sleep(1)

        # upload the sequence
        pulsedmasterlogic.sample_ensemble(exp_name,True)
        while pulsedmasterlogic.status_dict['sampload_busy'] or pulsedmasterlogic.status_dict['sampling_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:     
            time.sleep(1)

        #print('Finished uploading', dictparams_oc["name"], '!')

        # make sure everything is finished (crucial)
        #time.sleep(1)#time.sleep(5)    

         # start the measurement
        #print('Starting the measurement!')
        pulsedmasterlogic.toggle_pulsed_measurement(True)
        time.sleep(1)

        # make sure the measurement started
        while not pulsedmasterlogic.status_dict['measurement_running']:
            time.sleep(0.1)

        measurement_start_time = time.time()

        while time.time() <= measurement_start_time + runtime:
            time.sleep(0.2)

            # option to stop the measurement
            if uglobals.abort.is_set():
                print('Stopping the measurement!')
                break

        # option to stop the measurement
        if uglobals.abort.is_set():
            print('Stopping the measurement!')
            pulsedmasterlogic.toggle_pulsed_measurement(False)
            break

        #time.sleep(1)

        # Stop the measurement
        pulsedmasterlogic.toggle_pulsed_measurement(False)

        # Make sure it stopped (Wait until the Picoscope card sends the data)
        while pulsedmasterlogic.status_dict['measurement_running']:
            time.sleep(0.1)

        #######################################################################################################
        # Analysis
        ######################################################################################################

        fom_func = script_params['fom_function']
        foms.append(fom_func())

        idx += 1
except:
    logger.exception("")

In [951]:
idx
foms

[0.9594464623032213, 0.9749895179613619, 0.9728102331834259, 0.9924736426723433, 0.9864215896643189, 0.9730603077839967, 0.97476466944099, 0.9868381799747639, 0.9735556006467943, 0.9793450392334884, 0.9805427764166507, 0.9779736254138209, 0.9872446144440312, 0.9845148187950945, 0.9708480684024023, 0.9643792731916365, 0.9720278005060363, 0.9994402381002604, 0.9938555558829945, 0.9855089900145034]

In [957]:
np.mean(foms), np.std(np.asarray(foms))

(0.9795020502016067, 0.009808774306097796)

## Save the results

In [975]:
try:
    save_optimization_result(folder_path)
except:
    logger.exception("")

In [429]:
optimizer_obj

In [465]:
optimizer_obj.__dict__

{'interface_job_name': 'test_dCRAB_Noisy_2_control_fields', 'communication_obj': <quocslib.communication.AllInOneCommunication.AllInOneCommunication object at 0x000000BA25DA8F60>, 'results_path': 'C:\\Users\\Setup3-PC\\Desktop\\qudi\\QuOCS_Results\\20220722_001553_test_dCRAB_Noisy_2_control_fields', 'optimizer_attribute': <class 'quocslib.optimizationalgorithms.dCRABAlgorithm.dCRABAlgorithm'>, 'opt_alg_obj': <quocslib.optimizationalgorithms.dCRABAlgorithm.dCRABAlgorithm object at 0x000000BA253D3F98>}

In [495]:
pulses_list, time_grids_list, parameters_list = optimizer_obj.opt_alg_obj.get_best_controls()

t_amplitude = time_grids_list[0]
amplitude = pulses_list[0]
t_phase = time_grids_list[1]
phase = pulses_list[1]

In [981]:
best_dict = optimizer_obj.opt_alg_obj.get_best_controls()

pulses_list = best_dict['pulses']
time_grids_list = best_dict['timegrids']
parameters_list = best_dict['parameters']

pulses_list

In [561]:
optimizer_obj.opt_alg_obj.get_best_controls().keys()

dict_keys(['pulses', 'parameters', 'timegrids'])